In [ ]:

!pip3 install -U lightautoml

# QUICK WORKAROUND FOR PROBLEM WITH PANDAS
!pip3 install -U pandas

from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task

import torch

from category_encoders import WOEEncoder

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Downloading data**

In [ ]:
train_df = pd.read_csv('/kaggle/input/playground-series-s3e3/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s3e3/test.csv')
submission = pd.read_csv('/kaggle/input/playground-series-s3e3/sample_submission.csv')
addition_data = pd.read_csv('/kaggle/input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')

train_df['is_generated'] = 1
test_df['is_generated'] = 1
addition_data['is_generated'] = 0

In [ ]:
train_df = train_df.drop(527).drop(1398).reset_index(drop=True)

In [ ]:
train_df = train_df.drop('id', axis=1)
train_df

In [ ]:
train_df.Attrition.hist()

In [ ]:
train_df.isna().any()

In [ ]:
addition_data['Attrition'] = (addition_data['Attrition'] == 'Yes').astype(int)
# addition_data = addition_data[addition_data.Attrition == 1]
addition_data

In [ ]:
addition_data.Attrition.hist()

In [ ]:
addition_data.isna().any()

In [ ]:
train_df = pd.concat([train_df, addition_data],axis=0, ignore_index=True)
train_df = train_df.drop('EmployeeNumber', axis=1)
train_df

In [ ]:
train_df.isna().any()

In [ ]:
def is_young(x):
    if x <=25:
        return 1
    else:
        return 0
    
def young_and_low_daily_rate(x):
    if x['Age'] <= 25 & x['DailyRate'] < 500:
        return 1
    else:
        return 0
    
def overtime_satisfaction(x):
        if x['OverTime'] == 'Yes':
            return (x['MonthlyIncome'] * (x['StockOptionLevel'] + 0.05) * x['JobSatisfaction'])/x['Age']
        else:
            return (x['MonthlyIncome'] * (x['StockOptionLevel'] + 1.05) * x['JobSatisfaction'])/x['Age']

In [ ]:
df = pd.concat([train_df, test_df], axis=0)
df = df.drop(["EmployeeCount", "Over18", "StandardHours"], axis=1)

df['is_young'] = df['Age'].apply(lambda x: is_young(x))
df['young_and_underpaid'] = df.apply(lambda x: young_and_low_daily_rate(x), axis = 1)
df['worklife_stock'] = df.apply(lambda x: x['WorkLifeBalance'] + x['StockOptionLevel'], axis = 1)

df['income_satisfaction'] = df.apply(lambda x: x['JobSatisfaction'] * x['MonthlyIncome'], axis = 1)
df['income_level_environ_job_sat'] = df.apply(lambda x: x['EnvironmentSatisfaction']*x['JobSatisfaction'] * (x['MonthlyIncome']/x['JobLevel']), axis = 1)
df['overtime_stock'] = df.apply(lambda x: overtime_satisfaction(x), axis = 1)

# df = pd.get_dummies(df)
df

In [ ]:
features = ['Age', 'BusinessTravel', 'DailyRate', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'EnvironmentSatisfaction', 'Gender',
       'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobRole',
       'JobSatisfaction', 'MaritalStatus', 'MonthlyIncome', 'MonthlyRate',
       'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager', 'is_generated', 'id', 'is_young',
       'young_and_underpaid', 'worklife_stock', 'income_satisfaction',
        'income_level_environ_job_sat', 'overtime_stock']
cat_features = ['BusinessTravel', 'Department','Education', 'EducationField', 'EnvironmentSatisfaction', 'Gender',
               'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus','NumCompaniesWorked', 'OverTime', 
               'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 
                'WorkLifeBalance', 'YearsAtCompany','is_young', 'young_and_underpaid']

Woe encoder from @faelk8 notebook https://www.kaggle.com/code/faelk8/catboost/notebook

In [ ]:
woe = WOEEncoder(drop_invariant=True, randomized = True)
for col in cat_features:
    df[col] = df[col].astype(str)
woe.fit(df[features][:-len(test_df)], df['Attrition'][:-len(test_df)], cols = cat_features)
X = woe.transform(df[features])
X['Attrition'] = df['Attrition']
df = X

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

y = df['Attrition']
df = df.drop(['id', 'Attrition'], axis=1)

df[df.columns] = scaler.fit_transform(df[df.columns])

In [ ]:
train_df = df.iloc[:-len(test_df),:]
train_df['Attrition'] = y[:-len(test_df)]
test_df = df.iloc[-len(test_df):,:].reset_index(drop=True)

X = train_df.drop('Attrition', axis=1)
y = train_df.Attrition
X_test = test_df

**The model**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


In [ ]:
params = {
#     'metric': 'binary_logloss',
    'metric': 'auc',
#     'n_estimators': 10000,
    'objective': 'binary',
    'learning_rate': 0.02,
    'min_child_samples': 150,
    'reg_alpha': 3e-5,
    'reg_lambda': 9e-2,
    'num_leaves': 20,
    'max_depth': 16,
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'subsample_freq': 2,
    'max_bin': 240
}

cb_params = {
    'max_depth':6,
    'max_ctr_complexity': 5,
    'num_trees': 50000,
    'od_wait': 500,
    'od_type':'Iter', 
    'learning_rate': 0.04,
    'min_data_in_leaf': 3
}

In [ ]:
task = Task('binary', metric = 'auc')

N_THREADS = 4
N_FOLDS = 5
RANDOM_STATE = 42
TEST_SIZE = 0.2
TIMEOUT = 60 * 60
TARGET_NAME = 'Attrition'

np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)
# TabularUtilizedAutoML
# TabularAutoML
automl = TabularAutoML(
    task = task, 
    timeout = TIMEOUT,
    cpu_limit = N_THREADS,
    general_params = {'use_algos': [['linear_l2', 'lgb', 'lgb_tuned', 'cb']]},
    lgb_params = {'default_params': params, 'freeze_defaults': True},
    cb_params = {'default_params': cb_params, 'freeze_defaults': True},
#     reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE}
#     general_params = {"use_algos": [all_models]},
    reader_params = {'n_jobs': N_THREADS}
)

In [ ]:
roles = {
    'target': TARGET_NAME
}

In [ ]:
preds_tr = automl.fit_predict(train_df, roles = roles, verbose = 1)

In [ ]:
preds_tr

In [ ]:
preds = automl.predict(X_test)
preds

**Making submission**

In [ ]:
submission['Attrition'] = preds.data[:,0]
submission

In [ ]:
submission.to_csv('submission.csv', index=False)